In [ ]:
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
import numpy as np

In [ ]:
LFP = Structure.from_file("/data/kevinxhan/dist_chgnet/LiFePO4.cif")
LFP.make_supercell((8, 2, 2))
LFP_atoms = AseAtomsAdaptor().get_atoms(LFP)
print(len(LFP_atoms))

In [ ]:
import matgl
from DistMLIP.implementations.matgl import Potential_Dist, PESCalculator_Dist
from DistMLIP.implementations.matgl import CHGNet_Dist, TensorNet_Dist
from DistMLIP.implementations.matgl import MolecularDynamics, Relaxer


model = matgl.load_model('/data/kevinxhan/dist_chgnet/matgl-private/pretrained_models/TensorNet-MatPES-PBE-v2025.1-PES').model
dist_model = TensorNet_Dist.from_existing(model)
dist_model.enable_distributed_mode([5,7])
model.eval()
""



In [ ]:
pes = Potential_Dist(model=dist_model, num_threads=256)

driver = MolecularDynamics(
    LFP_atoms,
    potential=pes,
    timestep=0.5,
    temperature=300,
    # logfile="H2O_0.1M_1000k.log",
    # trajectory="H2O_0.1M_1000k.traj",
    loginterval=200
)

driver.run(20)